In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# 현재경로 확인
os.getcwd()

'D:\\포스코 아카데미\\새 폴더\\전체강의자료\\Part 05~11) Machine Learning\\08. Ensemble 기법의 종류와 원리\\실습코드'

In [3]:
# 데이터 불러오기
data = pd.read_csv("../Data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [4]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [6]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


## 의미가 없다고 판단되는 변수 제거

In [7]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models)

In [9]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x=sm.add_constant(train_x, has_constant="add")
sm_model=sm.OLS(train_y, sm_train_x)
fitted_sm_model=sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Wed, 10 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:45:24   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
sm_test_x=sm.add_constant(test_x, has_constant='add')
sm_model_predict=fitted_sm_model.predict(sm_test_x)

In [13]:
sqrt(mean_squared_error(sm_model_predict, test_y)) #RMSE

239804.29670858165

# Bagging 한 결과가 일반적인 결과보다 좋은지 확인

In [18]:
import random
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range(train_x.shape[0])]
    random_data_index=np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x=train_x.iloc[random_data_index,]
    sm_train_y=train_y.iloc[random_data_index,]
    sm_train_x = sm.add_constant(sm_train_x, has_constant = 'add')
    sm_model=sm.OLS(sm_train_y, sm_train_x)
    fitted_model=sm_model.fit()
    pred=fitted_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred, test_y)))

9547
241099.02814566964
9512
240152.0726266546
9554
240044.89025410902
9533
239701.1023926
9504
239746.22370899635
9587
239760.43615773707
9616
239970.01385595542
9589
240750.54106660667
9574
240238.18540533952
9597
239690.48200174858


In [20]:
bagging_predict_result[2]

735      5.551901e+05
2830     6.978490e+05
4106     1.113122e+06
16218    1.467922e+06
19964    6.946075e+05
             ...     
12606    5.945839e+05
14393    6.762002e+05
6899     3.306971e+05
85       9.100338e+05
21363    4.289182e+05
Length: 6484, dtype: float64

In [23]:
bagging_predict=[]
for lst2_index in range(test_x.shape[0]):
    temp_predict=[]
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [24]:
bagging_predict

[564128.122723806,
 712406.3555133644,
 1119907.4265045393,
 1476790.2261149702,
 699424.0430511914,
 386504.7048250325,
 788061.0805937711,
 485132.1561385868,
 499178.8113081615,
 538792.5776016881,
 644077.0717655227,
 410131.7405574948,
 267604.2275860514,
 279555.30519224604,
 338394.5468685607,
 1264202.767663515,
 324522.27655663126,
 1035326.2628111364,
 261587.6380389423,
 604419.5606385774,
 392167.7297173072,
 1302959.3164092854,
 822229.1595212739,
 582621.8008447883,
 596098.7002132668,
 570299.0237703237,
 265742.77829358366,
 41046.17306494914,
 564406.716172634,
 641883.7921687012,
 565149.8655576768,
 459639.5901851504,
 554288.9954336189,
 688211.8023289046,
 410066.63649323734,
 877618.3617109495,
 939699.5799067135,
 638280.5290765743,
 390966.9555691803,
 1081166.8258960058,
 453439.74748256465,
 148931.46868046332,
 489591.6975508242,
 219990.4252200479,
 65497.546550553816,
 -44249.19835742832,
 249192.59909038112,
 283230.66171793296,
 362969.77768819476,
 72202

In [26]:
sqrt(mean_squared_error(bagging_predict, test_y))

239725.76263734742

## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit-Learn)

In [27]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.2967085815


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [28]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239918.33035074198


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 그렇다면 Sampling을 많이 해보자!

In [29]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 239856.7503772534


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [30]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 293611.08864555875


In [31]:
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range(train_x.shape[0])]
    random_data_index=np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x=train_x.iloc[random_data_index,]
    sm_train_y=train_y.iloc[random_data_index,]
    decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
    tree_model1 = decision_tree_model.fit(sm_train_x, sm_train_y) # 학습 데이터를 의사결정나무 모형에 적합
    predict1 = tree_model1.predict(test_x)
    bagging_predict_result.append(predict1)
    print(sqrt(mean_squared_error(predict1, test_y)))

9565
295094.55750320834
9578
266427.4600275098
9528
306766.46975151234
9590
285423.7758242435
9529
284164.4123434742
9512
319908.6595277194
9573
287519.22112155514
9590
300325.3404718033
9574
304580.0044204881
9456
281180.5230364152


In [32]:
bagging_predict=[]
for lst2_index in range(test_x.shape[0]):
    temp_predict=[]
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index][lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [33]:
bagging_predict

[349500.0,
 672330.0,
 1015408.3333333334,
 1413000.0,
 611857.1152472527,
 407708.3333333334,
 846593.5714285715,
 441100.0,
 458327.0105782341,
 437542.5,
 595916.6666666667,
 420882.7678571429,
 289118.4375,
 379787.7045454546,
 364000.0,
 1238750.0,
 635059.4,
 1233400.0,
 324700.0,
 622602.0,
 347932.5,
 2167000.0,
 521775.0,
 406607.5,
 573417.6190476192,
 538250.0000000001,
 266421.21428571426,
 286560.0,
 362603.6416666667,
 583200.2,
 570485.0,
 652197.4333333333,
 547918.8333333333,
 520900.0,
 372404.1666666667,
 924660.0,
 1003950.0,
 614000.0,
 445300.0,
 1473550.0,
 501216.6666666666,
 279450.0,
 506395.33285714284,
 277860.6,
 259816.6666666667,
 202916.66666666666,
 273320.0,
 290980.9252702277,
 292502.0303030303,
 522360.0,
 377813.84280303033,
 366877.7666666667,
 666977.6,
 372741.0,
 462349.56160218094,
 2501250.0,
 509360.71428571426,
 723083.3333333333,
 343800.0,
 625293.5103383458,
 567607.6666666667,
 384718.3333333334,
 228365.5402930403,
 389307.75,
 428821.

In [34]:
sqrt(mean_squared_error(bagging_predict, test_y))

239873.5670053802

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [13]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 244509.51754833185


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [13]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 235007.94326063182


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
